In [ ]:
import os
import sys
import pandas as pd
from pynxtools_apm.utils.versioning import __version__
from pynxtools_apm.examples.get_sha256_of_directories import SEPARATOR
from pynxtools_apm.examples.oasisb_utils import APT_MIME_TYPES, CAMECA_ROOT_MIME_TYPES
from pynxtools_apm.examples.get_file_from_archive_formats import get_file_from_zip, get_file_from_tar, get_file_from_rar, get_file_from_sevenzip

# TODO parameterize
with open("root_directory.txt") as fp:
    root_directory = f"{fp.read().strip().replace('/', os.sep)}"
    src_directory = root_directory
    trg_directory = root_directory
print(src_directory)
print(trg_directory)

header = "file_path:archive_path;byte_size;unix_mtime;sha256sum"

In [ ]:
config: dict[str, str] = {
    "python_version": f"{sys.version.replace(' ', '_')}",
    "working_directory": f"{os.getcwd()}",
    "pynxtools_apm version": f"{__version__}",
    # "directory": f"src_directory,  # sys.argv[1],
}

ods = pd.read_excel(
    f"{src_directory}{os.sep}data{os.sep}aaa_legacy_data.ods",
    sheet_name="aaa_legacy_data",
    engine="odf",
)
# cnt = 0
for row in ods.itertuples(index=True):
    if row.project_name not in ("aus_sydney_bilal"):
        continue
    if row.parse in (1, 2):
        print(f"project{SEPARATOR}{row.project_name}{SEPARATOR}hashing...")
        sub_directory = f"{src_directory}{os.sep}data{os.sep}{row.project_name}"
        project_csv = (
            f"{src_directory}{os.sep}data{os.sep}{row.project_name}.sha256.results.csv"
        )
        with open(project_csv, "r") as fp:
            start = next(idx for idx, line in enumerate(fp) if header in line)
        df = pd.read_csv(project_csv, sep=";", skiprows=start)
        df.columns = ["path", "size", "mtime", "sha256"]
        print(df)

        # multiple checks for consistency
        # i) which files with atom-probe-specific mimetype are listed in df?
        file_to_hash: dict[str, str] = {}
        for line in df.itertuples(index=True):
            if line.path.lower().endswith(
                tuple(APT_MIME_TYPES + CAMECA_ROOT_MIME_TYPES)
            ):
                # ii) which of these are unique
                if line.path not in file_to_hash:
                    file_to_hash[line.path] = line.sha256
                    print(f"{line.path}")
                else:
                    print(f"{line.path} is a duplicate")
        # iii) are all the files that are referred to in each row, i.e., puzzled together workflow, in f"{row.project_name}.ods" listed in df?
        for line in df.itertuples(index=True):
            for col, value in zip(df.columns, row[1:]):  # skip index
                if value == "":
                    continue
                if value in file_to_hash:
                    continue
                print(f"{value} is a file that has not been hashed")
        # iv) unpack these files
        for row_idx in range(df.shape[0]):  # TODO ?? is the header skipped ?
            for col_idx in range(0, 6):  # TODO !!  df.shape[1])
                value = df.iat[row_idx, col_idx]
                if value == "":
                    continue
                else:
                    trg_file_name = f"{row.project_name}.{row_idx}.{col_idx}.{TODOHASH}.{value[value.rfind('.') + 1 :].lower()}"
                    if value.count(":") == 1:
                        tmp = value.split(":")
                        archiv_file_path = tmp[0]
                        file_path = tmp[1]
                        if archiv_file_path.lower().endswith((".zip", ".eln")):
                            get_file_from_zip(
                                archive_file_path,
                                file_path,
                                trg_directory,
                                trg_file_name,
                            )
                        elif archiv_file_path.lower().endswith(
                            (".tar", ".tar.gz", ".tar.bz2", ".tar.xz")
                        ):
                            get_file_from_tar(
                                archive_file_path,
                                file_path,
                                trg_directory,
                                trg_file_name,
                            )
                        elif archiv_file_path.lower().endswith((".rar")):
                            get_file_from_rar(
                                archive_file_path,
                                file_path,
                                trg_directory,
                                trg_file_name,
                            )
                        elif archiv_file_path.lower().endswith((".7z")):
                            get_file_from_sevenz(
                                archive_file_path,
                                file_path,
                                trg_directory,
                                trg_file_name,
                            )
                        del tmp, archiv_file_path, file_path
                    else:
                        shutil.copy2(
                            f"{src_directory}{os.sep}{data}{os.sep}{row.project_name}{value}",
                            f"{trg_directory}{os.sep}{trg_file_name}",
                        )
                    del trg_file_name
            # TODO DELs
        print(f"Row {row_idx}, Column {col_idx} = {value}")

        del project_csv, start, df
        # cnt = 0